In [5]:
                                                                                                                                                                                                                             import pandas as pd
import numpy as np
import os
import shutil
import glob
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from itertools import combinations
from joblib import Parallel, delayed, dump
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from pathlib import Path
from joblib import load
from sklearn.model_selection import GridSearchCV
from joblib import Parallel, delayed
import gc
from tqdm import tqdm
import torch
from sklearn.model_selection import KFold
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import joblib

In [12]:
def get_samples(vertices_folder, classes=None):
    sample_ids, labels = [], []
    for path in glob.glob(os.path.join(vertices_folder, '*_RL_*.pickle')):
        fname = os.path.basename(path)
        parts = fname.split('_')
        sample_id = parts[0]
        cls = parts[-1].split('.')[0]
        if classes is None or cls in classes:
            sample_ids.append(sample_id)
            labels.append(cls)
    return sample_ids, labels

In [13]:
all_ids, all_labels = get_samples(vertices_dir)

In [15]:
def load_all_data(vertices_folder, corr_folder, sample_ids, labels, n_vertices=379, dtype=np.float32):
    n_samples = len(sample_ids)
    edges = build_edge_list(n_vertices)
    n_edges = len(edges)

    means_all = np.zeros((n_samples, n_vertices), dtype=dtype)
    stds_all  = np.zeros((n_samples, n_vertices), dtype=dtype)
    weights_all = np.zeros((n_samples, n_edges), dtype=dtype)

    for idx, (sid, cls) in tqdm(enumerate(zip(sample_ids, labels))):
        vfile = os.path.join(vertices_folder, f"{sid}_RL_{cls}.pickle")
        df_v = pd.read_pickle(vfile)
        means_all[idx] = df_v['mean'].values.astype(dtype)
        stds_all[idx]  = df_v['std'].values.astype(dtype)

        corr_file = os.path.join(corr_folder, f"{sid}_RL_{cls}.pkl")
        df_c = pd.read_pickle(corr_file)
        df_c['i'] = df_c['vertex_1'].str[1:].astype(int)
        df_c['j'] = df_c['vertex_2'].str[1:].astype(int)
        df_c.sort_values(['i', 'j'], inplace=True)
        weights_all[idx] = df_c['weight'].values.astype(dtype)

    return means_all, stds_all, weights_all, edges



In [ ]:
def train_edge_models(means, stds, weights, y, n_jobs=-1):
    n_edges = weights.shape[1]
    coefs = np.zeros((n_edges, len(np.unique(y)), 5), np.float32)
    intercepts = np.zeros((n_edges, len(np.unique(y))), np.float32)

    results = Parallel(n_jobs=n_jobs)(
        delayed(fit_one_edge)(k, means, stds, weights, y)
        for k in tqdm(range(n_edges))
    )
    for k, coef, interc in results:
        coefs[k] = coef
        intercepts[k] = interc
    return coefs, intercepts

In [12]:
def build_edge_list(n_vertices=379):
    return [(i, j) for i in range(n_vertices) for j in range(i + 1, n_vertices)]

In [ ]:
def sample_probs_per_edge(mean, std, weights, edges, coefs, intercepts):
    X = np.stack([
        mean[[i for i,_ in edges]],
        std [[i for i,_ in edges]],
        mean[[j for _,j in edges]],
        std [[j for _,j in edges]],
        weights
    ], axis=1) 
    logits = (coefs * X[:,None,:]).sum(axis=2) + intercepts
    return scipy.special.softmax(logits, axis=1)  

In [ ]:
def build_vertex_features_for_sample(probs_per_edge, vertex_to_edges):
    n_classes = probs_per_edge.shape[1]
    feats = []
    for incident in vertex_to_edges:
        if len(incident)>0:
            vp = probs_per_edge[incident].mean(axis=0)
        else:
            vp = np.zeros(n_classes, dtype=probs_per_edge.dtype)
        feats.append(vp)
    return np.concatenate(feats)  

# OOF averaging for vertices, RL

In [ ]:

n_train = means_train.shape[0]
n_vertices = means_train.shape[1]

le = LabelEncoder()
y_train_enc = le.fit_transform(train_labels)

n_edge_classes = len(np.unique(y_train_enc))
n_vertex_feats = n_vertices * n_edge_classes
X_train_oof = np.zeros((n_train, n_vertex_feats), dtype=np.float32)

kf = KFold(n_splits=3, shuffle=True, random_state=42)
vertex_to_edges = build_vertex_edge_map(edges, n_vertices)

for train_idx, val_idx in tqdm(kf.split(range(n_train))):
    coefs_cv, intercepts_cv = train_edge_models(
        means_train[train_idx], stds_train[train_idx],
        weights_train[train_idx], y_train_enc[train_idx], n_jobs=-1
    )

    for idx in val_idx:
        mean, std, weights = means_train[idx], stds_train[idx], weights_train[idx]
        probs_edge = sample_probs_per_edge(mean, std, weights, edges, coefs_cv, intercepts_cv)
        X_train_oof[idx] = build_vertex_features_for_sample(probs_edge, vertex_to_edges)

In [23]:

coefs_full, intercepts_full = train_edge_models(
    means_train, stds_train, weights_train, y_train_enc, n_jobs=-1
)


100%|██████████| 71631/71631 [18:24<00:00, 64.86it/s]


In [26]:
clf_final = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=5000)
clf_final.fit(X_train_oof, y_train_enc)

LogisticRegression(max_iter=5000, multi_class='multinomial')

In [ ]:

X_test = []
for sid, lbl in zip(test_ids, test_labels):
    mean, std, weights = load_sample(sid, lbl, vertices_dir, corr_dir)
    probs_edge = sample_probs_per_edge(mean, std, weights, edges, coefs_full, intercepts_full)
    X_test.append(build_vertex_features_for_sample(probs_edge, vertex_to_edges))
X_test = np.vstack(X_test)
y_test_enc = le.transform(test_labels)

In [27]:
acc = (clf_final.predict(X_test) == y_test_enc).mean()
print(f"OOF-based test accuracy: {acc:.3f}")


OOF-based test accuracy: 0.955


# Averaging for vertices with entropy, RL

In [28]:
import numpy as np
import scipy.special
from collections import defaultdict

def sample_edge_entropies(mean, std, weights, edges, coefs, intercepts):
    X = np.stack([
        mean[[i for i,_ in edges]],
        std [[i for i,_ in edges]],
        mean[[j for _,j in edges]],
        std [[j for _,j in edges]],
        weights
    ], axis=1)  
    logits = (coefs * X[:, None, :]).sum(axis=2) + intercepts 
    probs = scipy.special.softmax(logits, axis=1)           
    eps = 1e-12
    ent = -np.sum(probs * np.log(probs + eps), axis=1)       
    return ent

def build_vertex_features_from_entropies(entropies, vertex_to_edges):
    feats = np.zeros(len(vertex_to_edges), dtype=entropies.dtype)
    for v, incident in enumerate(vertex_to_edges):
        if incident:
            feats[v] = entropies[incident].mean()
        else:
            feats[v] = 0.0
    return feats


In [ ]:
from sklearn.model_selection import KFold

n_train    = means_train.shape[0]
n_vertices = means_train.shape[1]

le = LabelEncoder()
y_train_enc = le.fit_transform(train_labels)

X_train_oof = np.zeros((n_train, n_vertices), dtype=np.float32)

kf = KFold(n_splits=3, shuffle=True, random_state=42)
vertex_to_edges = build_vertex_edge_map(edges, n_vertices)

for train_idx, val_idx in tqdm(kf.split(range(n_train))):
    coefs_cv, intercepts_cv = train_edge_models(
        means_train[train_idx],
        stds_train[train_idx],
        weights_train[train_idx],
        y_train_enc[train_idx],
        n_jobs=-1
    )

    for idx in val_idx:
        mean_i, std_i, w_i = (
            means_train[idx],
            stds_train[idx],
            weights_train[idx]
        )

        ent = sample_edge_entropies(
            mean_i, std_i, w_i,
            edges, coefs_cv, intercepts_cv
        )

        X_train_oof[idx] = build_vertex_features_from_entropies(
            ent, vertex_to_edges
        )


In [30]:
clf_final = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=5000)
clf_final.fit(X_train_oof, y_train_enc)

LogisticRegression(max_iter=5000, multi_class='multinomial')

In [31]:
n_test = len(test_ids)
X_test = np.zeros((n_test, n_vertices), dtype=np.float32)

for idx, (sid, lbl) in enumerate(zip(test_ids, test_labels)):
    mean_i, std_i, w_i = load_sample(sid, lbl, vertices_dir, corr_dir)
    ent = sample_edge_entropies(
        mean_i, std_i, w_i,
        edges, coefs_full, intercepts_full
    )
    X_test[idx] = build_vertex_features_from_entropies(ent, vertex_to_edges)

In [32]:
y_test_enc = le.transform(test_labels)
acc = (clf_final.predict(X_test) == y_test_enc).mean()
print(f"OOF-based test accuracy with entropy features: {acc:.3f}")


OOF-based test accuracy with entropy features: 0.712


# OOF averaging for vertices, LR

In [6]:
def get_samples(vertices_folder, classes=None):
    sample_ids, labels = [], []
    for path in glob.glob(os.path.join(vertices_folder, '*_LR_*.pickle')):
        fname = os.path.basename(path)
        parts = fname.split('_')
        sample_id = parts[0]
        cls = parts[-1].split('.')[0]
        if classes is None or cls in classes:
            sample_ids.append(sample_id)
            labels.append(cls)
    return sample_ids, labels

In [10]:
all_ids, all_labels = get_samples(vertices_dir)

In [7]:

def load_sample(sample_id, label, vertices_folder, corr_folder, dtype=np.float32):
    vfile = os.path.join(vertices_folder, f"{sample_id}_LR_{label}.pickle")
    df_v = pd.read_pickle(vfile)
    mean = df_v['mean'].values.astype(dtype)
    std  = df_v['std'].values.astype(dtype)

    cfile = os.path.join(corr_folder, f"{sample_id}_LR_{label}.pkl")
    df_c = pd.read_pickle(cfile)
    df_c['i'] = df_c['vertex_1'].str[1:].astype(int)
    df_c['j'] = df_c['vertex_2'].str[1:].astype(int)
    df_c.sort_values(['i', 'j'], inplace=True)
    weights = df_c['weight'].values.astype(dtype)

    return mean, std, weights




In [8]:
def load_all_data(vertices_folder, corr_folder, sample_ids, labels, n_vertices=379, dtype=np.float32):
    n_samples = len(sample_ids)
    edges = build_edge_list(n_vertices)
    n_edges = len(edges)

    means_all = np.zeros((n_samples, n_vertices), dtype=dtype)
    stds_all  = np.zeros((n_samples, n_vertices), dtype=dtype)
    weights_all = np.zeros((n_samples, n_edges), dtype=dtype)

    for idx, (sid, cls) in tqdm(enumerate(zip(sample_ids, labels))):
        vfile = os.path.join(vertices_folder, f"{sid}_LR_{cls}.pickle")
        df_v = pd.read_pickle(vfile)
        means_all[idx] = df_v['mean'].values.astype(dtype)
        stds_all[idx]  = df_v['std'].values.astype(dtype)

        corr_file = os.path.join(corr_folder, f"{sid}_LR_{cls}.pkl")
        df_c = pd.read_pickle(corr_file)
        df_c['i'] = df_c['vertex_1'].str[1:].astype(int)
        df_c['j'] = df_c['vertex_2'].str[1:].astype(int)
        df_c.sort_values(['i', 'j'], inplace=True)
        weights_all[idx] = df_c['weight'].values.astype(dtype)

    return means_all, stds_all, weights_all, edges



In [48]:
unique_ids = sorted(set(all_ids))
train_subj, test_subj = train_test_split(unique_ids, test_size=0.25, random_state=42)
train_ids = [sid for sid in all_ids if sid in train_subj]
train_labels = [lbl for sid, lbl in zip(all_ids, all_labels) if sid in train_subj]
test_ids = [sid for sid in all_ids if sid in test_subj]
test_labels = [lbl for sid, lbl in zip(all_ids, all_labels) if sid in test_subj]

In [51]:
means_train, stds_train, weights_train, edges = load_all_data(
    vertices_dir, corr_dir, train_ids, train_labels
)
le = LabelEncoder()
y_train_enc = le.fit_transform(train_labels)

6090it [08:05, 12.53it/s]


In [ ]:

n_train = means_train.shape[0]
n_vertices = means_train.shape[1]

le = LabelEncoder()
y_train_enc = le.fit_transform(train_labels)

n_edge_classes = len(np.unique(y_train_enc))
n_vertex_feats = n_vertices * n_edge_classes
X_train_oof = np.zeros((n_train, n_vertex_feats), dtype=np.float32)

kf = KFold(n_splits=3, shuffle=True, random_state=42)
vertex_to_edges = build_vertex_edge_map(edges, n_vertices)

for train_idx, val_idx in tqdm(kf.split(range(n_train))):
    coefs_cv, intercepts_cv = train_edge_models(
        means_train[train_idx], stds_train[train_idx],
        weights_train[train_idx], y_train_enc[train_idx], n_jobs=-1
    )

    for idx in val_idx:
        mean, std, weights = means_train[idx], stds_train[idx], weights_train[idx]
        probs_edge = sample_probs_per_edge(mean, std, weights, edges, coefs_cv, intercepts_cv)
        X_train_oof[idx] = build_vertex_features_for_sample(probs_edge, vertex_to_edges)

In [ ]:
clf_final = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=5000)
clf_final.fit(X_train_oof, y_train_enc)

In [ ]:

coefs_full, intercepts_full = train_edge_models(
    means_train, stds_train, weights_train, y_train_enc, n_jobs=-1
)


In [56]:

X_test = []
for sid, lbl in zip(test_ids, test_labels):
    mean, std, weights = load_sample(sid, lbl, vertices_dir, corr_dir)
    probs_edge = sample_probs_per_edge(mean, std, weights, edges, coefs_full, intercepts_full)
    X_test.append(build_vertex_features_for_sample(probs_edge, vertex_to_edges))
X_test = np.vstack(X_test)
y_test_enc = le.transform(test_labels)


acc = (clf_final.predict(X_test) == y_test_enc).mean()
print(f"OOF-based test accuracy: {acc:.3f}")


OOF-based test accuracy: 0.917


# LR, double-nested CV strategy

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import numpy as np


means_all, stds_all, weights_all, edges = load_all_data(
    vertices_dir, corr_dir, all_ids, all_labels
)
le = LabelEncoder()
y_all = le.fit_transform(all_labels)

K_outer = 4
outer_cv = KFold(n_splits=K_outer, shuffle=True, random_state=42)


In [14]:
n_vertices = means_all.shape[1]
num_edge_classes = len(np.unique(y_all))
meta_oof = np.zeros((len(all_ids), n_vertices * num_edge_classes), dtype=np.float32)

In [28]:

def build_vertex_edge_map(edges, n_vertices):
    vertex_to_edges = defaultdict(list)
    for k, (i, j) in enumerate(edges):
        vertex_to_edges[i].append(k)
        vertex_to_edges[j].append(k)
    return [vertex_to_edges[v] for v in range(n_vertices)]

In [30]:
n_vertices = means_all.shape[1]
vertex_to_edges = build_vertex_edge_map(edges, n_vertices)

In [ ]:
for outer_train_idx, outer_hold_idx in tqdm(outer_cv.split(all_ids)):
    m_tr, s_tr, w_tr = means_all[outer_train_idx], stds_all[outer_train_idx], weights_all[outer_train_idx]
    y_tr = y_all[outer_train_idx]
    K_inner = 4
    inner_cv = KFold(n_splits=K_inner, shuffle=True, random_state=42)
    oof_feats = np.zeros((len(outer_train_idx), n_vertices * num_edge_classes), dtype=np.float32)

    for inner_tr, inner_val in inner_cv.split(outer_train_idx):
        tr_idx = outer_train_idx[inner_tr]
        val_idx = outer_train_idx[inner_val]

        coefs, inters = train_edge_models(
            means_all[tr_idx], stds_all[tr_idx], weights_all[tr_idx], y_all[tr_idx]
        )


        for pos, idx in enumerate(val_idx):
            m, s, w = means_all[idx], stds_all[idx], weights_all[idx]
            probs = sample_probs_per_edge(m, s, w, edges, coefs, inters)
            oof_feats[np.where(outer_train_idx==idx)[0][0]] = \
                build_vertex_features_for_sample(probs, vertex_to_edges)

    
    meta_oof[outer_train_idx] = oof_feats
    coefs_full, inters_full = train_edge_models(
        m_tr, s_tr, w_tr, y_tr
    )

    for pos, idx in enumerate(outer_hold_idx):
        m, s, w = means_all[idx], stds_all[idx], weights_all[idx]
        probs = sample_probs_per_edge(m, s, w, edges, coefs_full, inters_full)
        meta_oof[idx] = build_vertex_features_for_sample(probs, vertex_to_edges)


In [33]:

clf_stack = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=5000)
clf_stack.fit(meta_oof, y_all)

oof_preds = clf_stack.predict(meta_oof)
acc = (oof_preds == y_all).mean()
print(f"LR tested accuracy: {acc:.3f}")


LR tested accuracy: 0.982


# RL, double-nested CV strategy

In [35]:
def get_samples(vertices_folder, classes=None):
    sample_ids, labels = [], []
    for path in glob.glob(os.path.join(vertices_folder, '*_RL_*.pickle')):
        fname = os.path.basename(path)
        parts = fname.split('_')
        sample_id = parts[0]
        cls = parts[-1].split('.')[0]
        if classes is None or cls in classes:
            sample_ids.append(sample_id)
            labels.append(cls)
    return sample_ids, labels


def load_sample(sample_id, label, vertices_folder, corr_folder, dtype=np.float32):
    vfile = os.path.join(vertices_folder, f"{sample_id}_RL_{label}.pickle")
    df_v = pd.read_pickle(vfile)
    mean = df_v['mean'].values.astype(dtype)
    std  = df_v['std'].values.astype(dtype)

    cfile = os.path.join(corr_folder, f"{sample_id}_RL_{label}.pkl")
    df_c = pd.read_pickle(cfile)
    df_c['i'] = df_c['vertex_1'].str[1:].astype(int)
    df_c['j'] = df_c['vertex_2'].str[1:].astype(int)
    df_c.sort_values(['i', 'j'], inplace=True)
    weights = df_c['weight'].values.astype(dtype)

    return mean, std, weights




In [38]:
def load_all_data(vertices_folder, corr_folder, sample_ids, labels, n_vertices=379, dtype=np.float32):
    n_samples = len(sample_ids)
    edges = build_edge_list(n_vertices)
    n_edges = len(edges)

    means_all = np.zeros((n_samples, n_vertices), dtype=dtype)
    stds_all  = np.zeros((n_samples, n_vertices), dtype=dtype)
    weights_all = np.zeros((n_samples, n_edges), dtype=dtype)

    for idx, (sid, cls) in tqdm(enumerate(zip(sample_ids, labels))):
        vfile = os.path.join(vertices_folder, f"{sid}_RL_{cls}.pickle")
        df_v = pd.read_pickle(vfile)
        means_all[idx] = df_v['mean'].values.astype(dtype)
        stds_all[idx]  = df_v['std'].values.astype(dtype)

        corr_file = os.path.join(corr_folder, f"{sid}_RL_{cls}.pkl")
        df_c = pd.read_pickle(corr_file)
        df_c['i'] = df_c['vertex_1'].str[1:].astype(int)
        df_c['j'] = df_c['vertex_2'].str[1:].astype(int)
        df_c.sort_values(['i', 'j'], inplace=True)
        weights_all[idx] = df_c['weight'].values.astype(dtype)

    return means_all, stds_all, weights_all, edges

In [40]:
means_all, stds_all, weights_all, edges = load_all_data(
    vertices_dir, corr_dir, all_ids, all_labels
)
le = LabelEncoder()
y_all = le.fit_transform(all_labels)

K_outer = 4
outer_cv = KFold(n_splits=K_outer, shuffle=True, random_state=42)


8134it [12:23, 10.93it/s]


In [41]:
n_vertices = means_all.shape[1]
num_edge_classes = len(np.unique(y_all))
meta_oof = np.zeros((len(all_ids), n_vertices * num_edge_classes), dtype=np.float32)

In [42]:

def build_vertex_edge_map(edges, n_vertices):
    vertex_to_edges = defaultdict(list)
    for k, (i, j) in enumerate(edges):
        vertex_to_edges[i].append(k)
        vertex_to_edges[j].append(k)
    return [vertex_to_edges[v] for v in range(n_vertices)]

In [43]:
n_vertices = means_all.shape[1]
vertex_to_edges = build_vertex_edge_map(edges, n_vertices)

In [ ]:
for outer_train_idx, outer_hold_idx in tqdm(outer_cv.split(all_ids)):
    m_tr, s_tr, w_tr = means_all[outer_train_idx], stds_all[outer_train_idx], weights_all[outer_train_idx]
    y_tr = y_all[outer_train_idx]
    K_inner = 4
    inner_cv = KFold(n_splits=K_inner, shuffle=True, random_state=42)
    oof_feats = np.zeros((len(outer_train_idx), n_vertices * num_edge_classes), dtype=np.float32)

    for inner_tr, inner_val in inner_cv.split(outer_train_idx):
        tr_idx = outer_train_idx[inner_tr]
        val_idx = outer_train_idx[inner_val]

        coefs, inters = train_edge_models(
            means_all[tr_idx], stds_all[tr_idx], weights_all[tr_idx], y_all[tr_idx]
        )

        for pos, idx in enumerate(val_idx):
            m, s, w = means_all[idx], stds_all[idx], weights_all[idx]
            probs = sample_probs_per_edge(m, s, w, edges, coefs, inters)
            oof_feats[np.where(outer_train_idx==idx)[0][0]] = \
                build_vertex_features_for_sample(probs, vertex_to_edges)


    meta_oof[outer_train_idx] = oof_feats
    coefs_full, inters_full = train_edge_models(
        m_tr, s_tr, w_tr, y_tr
    )
    for pos, idx in enumerate(outer_hold_idx):
        m, s, w = means_all[idx], stds_all[idx], weights_all[idx]
        probs = sample_probs_per_edge(m, s, w, edges, coefs_full, inters_full)
        meta_oof[idx] = build_vertex_features_for_sample(probs, vertex_to_edges)


In [45]:

clf_stack = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=5000)
clf_stack.fit(meta_oof, y_all)

oof_preds = clf_stack.predict(meta_oof)
acc = (oof_preds == y_all).mean()
print(f"RL tested accuracy: {acc:.3f}")


RL tested accuracy: 0.994
